In [2]:
!pip install --user -r requirements.txt

In [3]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
from wettbewerb import load_references, get_3montages
import mne
from scipy import signal as sig
import ruptures as rpt

In [4]:
training_folder = "../../shared_data/training_mini"

In [5]:
ids, channels, data, sampling_frequencies, reference_systems, eeg_labels = load_references(training_folder) 

100	 Dateien wurden geladen.


In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


In [11]:
fixed_length = 60000  # Angenommene feste Länge für jede Zeitreihe

X_processed = []

for montage_data in X:
    current_length = montage_data.shape[2]  # Die aktuelle Länge der Zeitreihe
    if current_length < fixed_length:
        # Auffüllen kürzerer Zeitreihen
        padding_length = fixed_length - current_length
        padding = np.zeros((montage_data.shape[0], montage_data.shape[1], padding_length))
        montage_data_padded = np.concatenate((montage_data, padding), axis=2)
    else:
        # Abschneiden, wenn die Aufnahme zu lang ist
        montage_data_padded = montage_data[:, :, :fixed_length]

    X_processed.append(montage_data_padded)

# Umwandeln in NumPy-Array und dann in PyTorch-Tensor
X_array = np.array(X_processed, dtype=np.float32)
X_tensor = torch.tensor(X_array)


In [13]:
fixed_length = 60000  # Angenommene feste Länge für jede Zeitreihe

X_processed = []

for i in range(len(data)):
    _, montage_data, _ = get_3montages(channels[i], data[i])
    current_length = montage_data.shape[1]

    if current_length < fixed_length:
        # Auffüllen kürzerer Zeitreihen
        padding = np.zeros((montage_data.shape[0], fixed_length - current_length))
        montage_data_padded = np.concatenate((montage_data, padding), axis=1)
    else:
        # Abschneiden, wenn die Aufnahme zu lang ist
        montage_data_padded = montage_data[:, :fixed_length]

    montage_data_reshaped = montage_data_padded.reshape(1, montage_data_padded.shape[0], montage_data_padded.shape[1])
    X_processed.append(montage_data_reshaped)

X_tensor = torch.tensor(np.array(X_processed), dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.long)

dataset = TensorDataset(X_tensor, Y_tensor)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [14]:
print("Form der Feature-Daten (X_tensor):", X_tensor.shape)
print("Form der Label-Daten (Y_tensor):", Y_tensor.shape)

# Anzeigen einiger Beispieldaten
print("Einige Beispieldaten:\n", X_tensor[0:5])

# Statistiken der Daten
print("Maximaler Wert in den Daten:", torch.max(X_tensor))
print("Minimaler Wert in den Daten:", torch.min(X_tensor))
print("Durchschnittlicher Wert in den Daten:", torch.mean(X_tensor.float()))

# Verteilung der Klassen
unique, counts = torch.unique(Y_tensor, return_counts=True)
print("Verteilung der Klassen in Y_tensor:", dict(zip(unique.tolist(), counts.tolist())))


Form der Feature-Daten (X_tensor): torch.Size([100, 1, 3, 60000])
Form der Label-Daten (Y_tensor): torch.Size([100])
Einige Beispieldaten:
 tensor([[[[-5.6763e-05, -5.1575e-05, -5.7373e-05,  ..., -1.1932e-03,
           -1.1874e-03, -1.1691e-03],
          [ 7.0190e-06,  3.3569e-05,  4.1504e-05,  ..., -5.7068e-05,
           -4.4556e-05,  1.8311e-06],
          [-5.4932e-06, -2.2888e-05, -1.5564e-05,  ..., -3.7842e-05,
           -6.3171e-05, -8.3008e-05]]],


        [[[ 3.6926e-05,  4.8523e-05,  4.6387e-05,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-5.4932e-06,  3.3264e-05,  3.9673e-05,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-2.7466e-06, -3.5706e-05, -3.6316e-05,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]]],


        [[[-3.1128e-05, -3.1433e-05, -3.6621e-05,  ...,  8.1177e-05,
            9.1858e-05,  9.1248e-05],
          [ 2.8076e-05,  5.0659e-05,  2.0447e-05,  ...,  7.4463e-05,
            1.1688e-04,  1.4008e-0

In [27]:
import torch
import torch.nn as nn

# Definieren Sie Ihr CNN-Modell
class CNN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=6, kernel_size=5),
            nn.BatchNorm1d(num_features=6),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(6, 16, 5),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2, 2),
        )
        
        # Berechnen der Größe für die erste vollverbundene Schicht
        with torch.no_grad():
            self._to_linear = None
            self._forward_conv(torch.zeros(1, *input_size))

        self.fc = nn.Sequential(
            nn.Linear(self._to_linear, 120),
            nn.BatchNorm1d(120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.ReLU(),
            nn.Linear(84, 2),  # 2 Ausgabeneuronen für binäre Klassifikation
        )

    def _forward_conv(self, x):
        x = self.classifier(x)
        if self._to_linear is None:
            self._to_linear = x.view(1, -1).size(1)
        return x

    def forward(self, x):
        x = self._forward_conv(x)
        x = x.view(-1, self._to_linear)  # Flatten
        x = self.fc(x)
        return x

# Erstellen Sie eine Instanz des Modells
input_size = (1, 60000)  # Angenommen, die Länge der Zeitreihe ist 60000
model = CNN(input_size)


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Definieren Sie die Verlustfunktion und den Optimierer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Definieren Sie die Anzahl der Epochen und den Batch-Size
num_epochs = 10
batch_size = 32

# Erstellen Sie ein DataLoader für die Trainingsdaten
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Trainings-Schleife
for epoch in range(num_epochs):
    model.train()  # Setzen des Modells in den Trainingsmodus
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        # Nullen Sie die Gradienten
        optimizer.zero_grad()
        
        # Vorwärtsdurchlauf
        outputs = model(inputs)
        
        # Berechnen Sie den Verlust
        loss = criterion(outputs, labels)
        
        # Rückwärtsdurchlauf und Optimierung
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Durchschnittlicher Verlust pro Epoch
    avg_loss = running_loss / len(train_loader)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}] - Loss: {avg_loss:.4f}')

print('Training abgeschlossen.')


RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [32, 1, 3, 60000]